# Install Libraries

In [9]:
!pip install transformers datasets huggingface_hub pydub numba librosa soundfile

  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached numba-0.59.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl (1.2 MB)
  Using cached safetensors-0.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached llvmlite-0.42.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached pooch-1.8.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached soxr-0.3.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached lazy_loader-0.3-py3-none-any.whl.metadata (4.3 kB)
Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
Using cached numba-0.59.0-cp310-cp310-manylinux20

In [32]:
!pip install boto3 ffmpeg

  Using cached ffmpeg-1.4-py3-none-any.whl


In [4]:
from huggingface_hub import login
login()

In [2]:
pip install --upgrade evaluate jiwer

  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached huggingface_hub-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached aiohttp-3.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 730.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━

# Download Dataset

In [4]:
from datasets import load_dataset

spgi1 = load_dataset("kensho/spgispeech", "test", use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for kensho/spgispeech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kensho/spgispeech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [5]:
print(spgi1)

DatasetDict({
    test: Dataset({
        features: ['wav_filename', 'audio', 'wav_filesize', 'transcript'],
        num_rows: 39341
    })
})


In [6]:
from tqdm import tqdm

# Sagemaker Whisper Model to Transcribe

In [7]:
import boto3, json
def query_endpoint(endpoint_name,body, content_type):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType=content_type, Body=body)
    model_predictions = json.loads(response['Body'].read())
    text = model_predictions['text']
    return text

In [4]:
endpoint_name='speech-to-text-meetsummarizar-endpoint'
y_actual_path = "./huggingFaceOutput/y_actual.txt"
y_pred_path = "./huggingFaceOutput/y_pred.txt"

In [ ]:
for i in tqdm(range(0,5000)):
    temp_audio_path = spgi1["test"][i]['audio']['path']
    y = spgi1["test"][i]['transcript']
    with open(temp_audio_path, "rb") as file:
        wav_file_read = file.read()
    temp_output = query_endpoint(endpoint_name,wav_file_read , "audio/wav")
    with open(y_actual_path, "a") as file:
        file.write(str(y) + "\n")
    with open(y_pred_path, "a") as file:
        file.write(str(temp_output[0].strip()) + "\n")

  1%|▏         | 70/5000 [01:46<2:07:10,  1.55s/it]

## Evaluate

In [5]:
with open(y_actual_path, "r") as file:
    actual_transcribe = file.read()
with open(y_pred_path, "r") as file:
    predicted_transcribe = file.read()

In [28]:
from evaluate import load
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()
wer_metric = load("wer")

wer = wer_metric.compute(references=[normalizer(actual_transcribe)], predictions=[normalizer(predicted_transcribe)])

print(wer)

0.08452891278671193


In [29]:
print((1-wer)*100)

91.54710872132881


# AWS Transcribe service to transcribe data

In [18]:
import os
def invoke_transcribe(path,job_name,format, output_bucket, output_key):
    client = boto3.client('transcribe')
    _ = client.start_transcription_job(
    TranscriptionJobName=job_name,
    MediaFormat=format,
    Media={
        'MediaFileUri': path
    },
    OutputBucketName=output_bucket,
    OutputKey=output_key,
    IdentifyMultipleLanguages=True)
def upload_to_s3(local_file, key, bucket="speech-to-text-meetsummarizar"):
    s3_client = boto3.client("s3")
    s3_client.upload_file(local_file, bucket, key)

In [19]:
batch_size = 100
wav_file_path = "./huggingFaceOutput/"
s3_bucket = "speech-to-text-meetsummarizar"
subbucket = "testing"
y_path_transcribe = "./huggingFaceOutput/y_actual_transcribe.txt"
from pydub import AudioSegment
for i in tqdm(range(0,int(5000/batch_size))):
    wav_file_array = []
    combined_audio = AudioSegment.silent(duration=0)
    y = ""
    for j in range(0,batch_size):
        temp_path = spgi1["test"][(batch_size*i)+j]['audio']['path']
        y = y + " " + spgi1["test"][(batch_size*i)+j]['transcript']
        wav_file_array.append(temp_path)
    with open(y_path_transcribe, "a") as file:
        file.write(str(y) + "\n")
    for file in wav_file_array:
        audio_segment = AudioSegment.from_wav(file)
        combined_audio += audio_segment
    input_path = wav_file_path+str(i)+".wav"
    output_s3_path = os.path.join(subbucket,str(i)+".wav")
    output_s3_text_path = os.path.join(subbucket,str(i)+".txt")
    combined_audio.export(input_path, format="wav")
    upload_to_s3(input_path, output_s3_path)
    os.remove(input_path)
    invoke_transcribe(path="s3://"+os.path.join(s3_bucket,output_s3_path),
                                                job_name=str(i)+"_job",
                                                format="wav", 
                                                output_bucket=s3_bucket, 
                                                output_key=output_s3_text_path)

100%|██████████| 50/50 [02:18<00:00,  2.77s/it]


In [22]:
print(temp)

[' This product continues to be very well received by users. Daily active user is approaching 10 million.']


In [33]:
import boto3
def download_from_s3(key, local_file, bucket="speech-to-text-meetsummarizar"):
    s3_client = boto3.client("s3")
    s3_client.download_file(bucket, key, local_file)

In [37]:
import os
import json
subbucket = "testing"
local_path = "./huggingFaceOutput"
output_file = "y_pred_transcribe.txt"
for i in range(50):
    file_name = str(i)+".txt"
    download_from_s3(os.path.join(subbucket,file_name),
                     os.path.join(local_path,file_name))
    with open(os.path.join(local_path,file_name), 'r') as file:
        obj = json.loads(file.read())
        content = " "+obj["results"]["transcripts"][0]["transcript"]
        with open(os.path.join(local_path,output_file), 'a') as new_file:
            new_file.write(str(content) + "\n")
    os.remove(os.path.join(local_path,file_name))

# Evaluate

In [38]:
y_actual_path_transcribe = "./huggingFaceOutput/y_actual_transcribe.txt"
y_pred_path_transcribe = "./huggingFaceOutput/y_pred_transcribe.txt"
with open(y_actual_path_transcribe, "r") as file:
    actual_transcribe_aws = file.read()
with open(y_pred_path_transcribe, "r") as file:
    predicted_transcribe_aws = file.read()
    
wer = wer_metric.compute(references=[normalizer(actual_transcribe_aws)], predictions=[normalizer(predicted_transcribe_aws)])

print(wer)

0.0600969678948716


In [39]:
print((1-wer)*100)

93.99030321051283
